# SPADE
SPADE *(Smart Python Agent Development Platform)* es una plataforma para el desarrollo de sistemas multiagante escrita en Python y basada en el protocolo [XMPP](http://www.xmpp.org/) para el intercambio de mensajes.



La documentación del proyecto y las instrucciones para su descarga están disponibles en

https://spade-mas.readthedocs.io/


# Instalación y preparación

La forma más rápida de instalar SPADE es usando `pip`.

In [ ]:
!pip install spade

Además de hacerlo aquí (para la ejecución de los agentes), tendrás que instalarlo en tu máquina para poder ejecutar el servidor XMPP (no se puede hacer desde Google Collab).

Arranca el servidor con `spade run` y apunta la IP para registrar tus agentes en el servidor cuando los crees.



## Creación de un agente simple

Tus agentes deben heredar de la clase `spade.agent.Agent`

Se ejecutan de forma asíncrona a través de [Async IO](https://realpython.com/async-io-python/), así que los métodos deben ir precedidos por la palabra `async` y las llamadas con `await`, que se encarga de lanzarlas y esperar sin bloquearse hasta que se obtiene el resultado.

IMPORTANTE: asyncIO proporciona un esquema de programación concurrente, pero no es ejecución real en paralelo. Las rutinas definidas como asíncronas se deben suspender de forma explícita (con `sleep`) cuando se encuentran inactivas, dejando que otras tareas se ejecutan mientras tanto.

El siguiente código define un agente que simplemente muestra un mensaje en la pantalla con su nombre. Se consigue sobreescribiendo el método `setup()` que se ejecuta cuando arranca el agente.

In [1]:
import spade

class DummyAgent(spade.agent.Agent):
  async def setup(self):
    print("Hello world! I'm agent {}".format(str(self.jid)))

`jid` es el identificador con el que se registra en el servidor XMPP.

Para crear el agent, hay que pasarle el nombre (sustituye la IP 127.0.0.1 por la del servidor XMPP o te dará un error de conexión) y un *password*. Para ejecutarlo usa `start()` y para detenerlo `stop()`.

In [2]:
dummy = DummyAgent("dummy@127.0.0.1", "1234")
await dummy.start()
print("Agent active: {}".format(dummy.is_alive()))
print("stopping...")
await dummy.stop()

print("Agent active: {}".format(dummy.is_alive()))

Hello world! I'm agent dummy@127.0.0.1
Agent active: True
stopping...
Agent active: False


## Agente con un comportamiento

La forma natural de programar agentes en SPADE es utilizando comportamientos. Es la forma de definir la respuesta del agente ante determinados eventos. SPADE dispone de varios comportamientos predefinidos, que se pueden extender con nuevos comportamientos propios si es necesario.

Uno de los más simples es el comportamiento cíclico, que se ejecuta de forma indefinida en bucle. El comportamiento tiene tres métodos especiales que se deben sobrecargar:

- `on_start()` se emplea para la inicialización del comportamiento y se ejecuta una sola vez cuando arranca
- `on_end()` se invoca cuando se detiene la ejecución del agente por completo (cuando se le "mata" o termina)
- `run()` es el método principal en el que se codifica lo que debe hacer el agente en cada iteración del ciclo. Recuerda que debe ser una corrutina asíncrona

Para usarlo, crea una clase comportamiento que herede de `CyclicBehaviour`. Por ejemplo, el siguiente comportamiento crea un contador y detiene el agente cuando llega a 5. En `on_start()` creamos el contador y lo inicializamos a cero. En `run()` se muestra su valor, se incrementa y se suspende el agente durante un segundo.

Podemos suspender un comportamiento llamando a `kill()`. Ten en cuenta que esto solo detiene el comportamiento, no al agente completo.


In [4]:
import asyncio
import spade
from spade import wait_until_finished
from spade.agent import Agent
from spade.behaviour import CyclicBehaviour

class CycBehav(CyclicBehaviour):
    async def on_start(self):
        print("CycBehav starts")
        self.counter = 0
        
    async def run(self):
        print("Counter: {}".format(self.counter))
        self.counter += 1
        if self.counter == 5:
            self.kill()
        await asyncio.sleep(1)
        
    async def on_end(self):
        print("CycBehav finished")

Para crear un agente que contenga ese comportamiento, lo añadiremos en el `setup()` del agente. Cuando el agente se ponga en marcha, activará automáticamente el comportamiento

In [5]:
class CycAgent(Agent):
    async def setup(self):
        print("Adding cyclic behaviour...")
        b = CycBehav()
        self.add_behaviour(b)
        print("...done. Agent starting")
    

En lugar de detenerlo, la función `wait_until_finished()` se queda esperando al agente hasta que termina el comportamiento especificado, o se le detiene desde fuera (con CTRL+C o kill directamente)

In [6]:
cyclic = CycAgent("cyclic@127.0.0.1", "1234")
await cyclic.start()
await wait_until_finished(cyclic)


Adding cyclic behaviour...
...done. Agent starting
CycBehav starts
Counter: 0
Counter: 1
Counter: 2
Counter: 3
Counter: 4
CycBehav finished


CancelledError: 

## Lanzamiento de agentes

Es posible crear y lanzar a ejecución un agente desde otro. Esto nos permite, por ejemplo, crear equipos con un leader que los coordina. Para ello vamos a usar otro de los comportamientos básicos: `OneShotBehaviour`, que se lanza una sola vez.

Para empezar, vamos a crear el agente

In [7]:
import spade
from spade.agent import Agent
from spade.behaviour import OneShotBehaviour

class SampleAgent(Agent):
    async def setup(self):
        print(f"{self.jid} created.")

A continuación creamos el comportamiento OneShot que se encargará de crear y lanzar otro agente igual

In [8]:
class LaunchBehav(OneShotBehaviour):
    async def run(self):
        agent2 = SampleAgent("follower@127.0.0.1", "1234")
        await agent2.start(auto_register=True)

Y por último creamos el agente principal que incorpora ese  comportamiento. Cuando arranca, lanza al segundo agente.

In [10]:
leader = SampleAgent("leader@127.0.01", "1234")
behav = LaunchBehav()
leader.add_behaviour(behav)
await leader.start(auto_register=True)

# wait until the behaviour is finished to quit spade.
await behav.join()

leader@127.0.01 created.
follower@127.0.0.1 created.


# Comunicación entre agentes

Una de las características más importantes de los agentes es su capacidad de comunicarse. Los agentes SPADE implementan el protocolo de comunicaciones de FIPA y la forma de gestionar los mensajes es a través de *templates*: unas plantillas que nos facilitan la gestión de los mensajes por parte de los comportamientos. Los atributos de un mensaje son
- to: el jid del receptor del mensaje
- sender: el jid del agente que lo envia
- body: el contenido del mensaje
- thread: un id que identifica el hilo de la conversación (u nagente puede mantener varias conversaciones de manera simultánea)
- metadata: un diccionario de pares (clave, valor) que incluye inofmración importante como la perfomrativa o la ontología que se está empleando.
Cuando un agente esté esperando un tipo determinado de mensaje, creará una plantilla que concida con él y la registrará. La plataforma se encarga de avisarle cuando llegue un mensaje que coincida con la plantilla.

Vamos a crear un comportamiento de tiempo OneShot para enviar un mensaje. Para eso, se crea una instancia de la clase `Message`, se rellena con la información y se envía con `send()` 

In [11]:
import spade
from spade.agent import Agent
from spade.behaviour import OneShotBehaviour
from spade.message import Message
from spade.template import Template


class InformBehav(OneShotBehaviour):
    async def run(self):
        print("InformBehav running")
        msg = Message(to="receiver@127.0.0.1")     # Instantiate the message
        msg.set_metadata("performative", "inform")  # Set the "inform" FIPA performative
        msg.body = "Hello World"                    # Set the message content

        await self.send(msg)
        print("Message sent!")

        # set exit_code for the behaviour
        self.exit_code = "Job Finished!"

        # stop agent from behaviour
        await self.agent.stop()
        
        
class SenderAgent(Agent):
    async def setup(self):
        print("SenderAgent started")
        self.b = InformBehav()
        self.add_behaviour(self.b)

El comportamiento para recibirlos es el recíproco, quedándose a la espera en un método `receive()`

In [12]:

class RecvBehav(OneShotBehaviour):
    async def run(self):
        print("RecvBehav running")

        msg = await self.receive(timeout=10) # wait for a message for 10 seconds
        if msg:
            print("Message received with content: {}".format(msg.body))
        else:
            print("Did not received any message after 10 seconds")

        # stop agent from behaviour
        await self.agent.stop()
        
        
class ReceiverAgent(Agent):
    async def setup(self):
        print("ReceiverAgent started")
        b = RecvBehav()
        template = Template()
        # here defines the template with the type of message it expects (an inform)
        template.set_metadata("performative", "inform")
        self.add_behaviour(b, template)

Solo queda crear los dos agentes. Cuando se inicien, automáticamente se activará el comportamiento para intercambiar un mensaje

In [13]:
receiveragent = ReceiverAgent("receiver@127.0.0.1", "1234")
await receiveragent.start(auto_register=True)
print("Receiver started")

senderagent = SenderAgent("sender@127.0.0.1", "1234")
await senderagent.start(auto_register=True)
print("Sender started")

await spade.wait_until_finished(receiveragent)
print("Agents finished")

ReceiverAgent started
Receiver started
RecvBehav running
SenderAgent started
Sender started
InformBehav running
Message sent!
Message received with content: Hello World
Agents finished


Si un agente quiere responder a otro, el método `make_reply()` de la clase Message se encarga de formatar directamente un mensaje de respuesta y solo hace falta rellenar el contenido o los campos que el agente necesite completar. 

In [ ]:
class ReceiveBehaviour(OneShotBehaviour):
    async def run(self):
        msg = await self.receive(timeout=10)
        if msg:
            reply = msg.make_reply()
            reply.body = "my reply"
            await self.send(reply)


# Ejercicio 1

Crea un agente que proponga a otros tres que le envíen un dato (un núnero entero) y el agente que lo propuso simplement elos guarda en una lista y los imprime. Usa una performativa de tipo "propose" (el cuerpo del mensaje de la propuesta no es relevante en este caso)

In [ ]:
#TODO: Añade tu código

# SPADE-BDI

Con SPADE es posible crear agentes que siguen la arquitectura BDI. Está desarrollado como un plugin. Al instalarse ya incluye una plataforma SPADE completa, por lo que si vas a usar este tipo de agentes puedes instalar directamente esta versión

In [ ]:
!pip install spade_bdi

Aunque se pueden añadir las creencias y el resto de elementos de al arquitectura usando sus correspondientes métodos, lo habitual es cargar un ficehro .asl que contenga el código en lenguaje AgentSpeak. 

Por ejemplo, vamos a generar un fichero que contiene las reglas para un hola mundo

In [14]:
with open('hello.asl', 'w') as fp:
    fp.write('!hello_world.\n')
    fp.write('+!hello_world <-\n')
    fp.write('    .print("Hello world!").\n')

In [15]:
!cat hello.asl

!hello_world.
+!hello_world <-
    .print("Hello world!").


Los agentes extendrán la clase BDIAgent, y en su creación habrá que indicar el fichero .asl que contiene su especificaicón

In [16]:
import asyncio
import spade
from spade_bdi.bdi import BDIAgent

a = BDIAgent("hello@127.0.0.1", "1234", "hello.asl")
await a.start()
#sleep the agent to let the rule to execute
await asyncio.sleep(1)
print("Agent ready")
await a.stop()   

hello@127.0.0.1 Hello world!
Agent ready


## Gestión de creencias

SPADE-BDI proporciona algunos métodos para revisar las creencias de los agentes, y también para añadir nuevas o eliminarlas.

In [17]:
with open('car.asl', 'w') as fp:
    fp.write('!start.\n')
    fp.write('+!start <-\n')
    fp.write('    +car(red);\n')
    fp.write('    +truck(blue).\n')

    fp.write('+car(Color)\n') 
    fp.write('    <- .print("The car is ",Color).!start.\n')

In [18]:
import asyncio
import time
import spade
from spade_bdi.bdi import BDIAgent

a = BDIAgent("car@127.0.0.1", "1234", "car.asl")
await a.start()
await asyncio.sleep(1)

print("Comenzando a interactuar con el comportamiento BDI del agente ... ");
# add a new belief
a.bdi.set_belief("car", "blue", "big")
await asyncio.sleep(1) #gives time to update belief set (comment and check the result)

# shows the belief set of the agent              
a.bdi.print_beliefs()
print("GETTING FIRST CAR BELIEF")
print("Primera Creencia de tipo car: ", a.bdi.get_belief("car"))
a.bdi.print_beliefs()
# removes a belief
a.bdi.remove_belief("car", 'blue', "big")
#without delay to remove the belief
a.bdi.print_beliefs()
print(a.bdi.get_beliefs())
a.bdi.set_belief("car", 'yellow')

time.sleep(1)
await a.stop()   

car@127.0.0.1 The car is  red
car@127.0.0.1 The car is  red
Comenzando a interactuar con el comportamiento BDI del agente ... 
car(red)
truck(blue)
car(blue, big)
GETTING FIRST CAR BELIEF
Primera Creencia de tipo car:  car(red)
car(red)
truck(blue)
car(blue, big)
car(red)
truck(blue)
car(blue, big)
['car(red)', 'truck(blue)', 'car(blue, big)']


## Creación de acciones internas

Además de las acciones predefinias que incorpora AgentSpeak, podemos escribir nuestras propias  funciones en python y utilizarlas desde los planes 

In [1]:
with open('actions.asl', 'w') as fp:
    fp.write('!start.\n')
    fp.write('+!start <-\n')
    fp.write('    .my_function(4, R);\n')
    fp.write('    .my_action(R).\n')

Hay dos funciones .my_function y .my_action que vamos a implementar directamente como métodos del agente

In [2]:
import asyncio
import argparse
import agentspeak
import spade
from spade_bdi.bdi import BDIAgent


class MyCustomBDIAgent(BDIAgent):
    def add_custom_actions(self, actions):
        
        @actions.add_function(".my_function", (int,))
        def _my_function(x):
            return x * x

        @actions.add(".my_action", 1)
        def _my_action(agent, term, intention):
            arg = agentspeak.grounded(term.args[0], intention.scope)
            print(arg)
            yield

In [3]:
a = MyCustomBDIAgent("actions@127.0.0.1", "1234", "actions.asl")

await a.start()
await asyncio.sleep(2)
await a.stop()

16


## Interfaz web para la monitorización

SPADE puoede generar una págian web que permite la monitorización en tiempo real de los agentes. Basta con indicarle la dirección de la máquina y el puerto al que conectarse. La web estará diponible en la url `ip:puerto/spade`

In [4]:
import asyncio
import argparse
import agentspeak
import spade
from spade_bdi.bdi import BDIAgent

ag = MyCustomBDIAgent("bdi@127.0.0.1","1234","actions.asl")
await a.start()

#version web
a.web.start(hostname="127.0.0.1",port="10000")
await asyncio.sleep(2000)
await a.stop()

La interfaz del agente está disponible en http://127.0.0.1:10000/spade

## Ejercicio 2

Incorpora la interfaz web a los agentes que intercambian mensajes del ejercicio 1 y comrpueba su funcionamiento

In [ ]:
#TODO: añade tu código

## Ejercicio 3

Implementa con SPADE-BDI agentes con el código de AgentSpeak de los ejercicios planteados (factorial o el robot limpiador

In [ ]:
#TODO: añade tu código